[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/Isotr0py/SakuraLLM-Notebooks/blob/main/Sakura-13B-Galgame-Kaggle-llama.cpp.ipynb)

In [ ]:
!git clone https://github.com/SakuraLLM/Sakura-13B-Galgame.git
%cd Sakura-13B-Galgame
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -q llama-cpp-python
!pip install llama-cpp-python --prefer-binary --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu118
!pip install -q -r requirements.llamacpp.txt
!pip install -q pyngrok

# install npm and localtunnel
!sudo apt-get install -qq -y nodejs
!npm install -g localtunnel
!rm -f /opt/conda/bin/node

In [ ]:
# ngrokToken留空则使用localtunnel进行内网穿透
ngrokToken = ""
MODEL = "sakura-13b-lnovel-v0.9-Q8_0"


from huggingface_hub import hf_hub_download
from pathlib import Path

if ngrokToken:
    from pyngrok import conf, ngrok
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(5000)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)
else:
    import subprocess
    import threading
    def start_localtunnel(port):
        p = subprocess.Popen(["lt", "--port", f"{port}"], stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')
    threading.Thread(target=start_localtunnel, daemon=True, args=(5000,)).start()

MODEL_PATH = f"./models/{MODEL}.gguf"
if not Path(MODEL_PATH).exists():
    hf_hub_download(repo_id="SakuraLLM/Sakura-13B-LNovel-v0.9-GGUF", filename=f"{MODEL}.gguf", local_dir="models/")

!python server.py \
    --model_name_or_path $MODEL_PATH \
    --llama_cpp \
    --use_gpu \
    --model_version 0.9 \
    --trust_remote_code \
    --no-auth